In [1]:
from qiskit import QuantumCircuit, transpile, Aer, IBMQ,execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator


# Loading your IBM Quantum account(s)
token = '01d110f88ee7d5cdfa271e96b5406f92065daa7c967a622b145d0560982310320f66627dc4d02a8d8c114a785a558d4d50ebde4f3540a0cbb5899809f0249d92'
IBMQ.save_account(token, hub='ibm-q-skku', group='skku', project='skku-students')
provider = IBMQ.get_provider(hub='ibm-q-skku', group='skku', project='skku-students')
provider = IBMQ.load_account()
provider.backends()

configrc.store_credentials:WARNING:2022-06-28 14:21:48,594: Credentials already present. Set overwrite=True to overwrite.


IBMQProviderError: 'No provider matches the specified criteria: hub = ibm-q-skku, group = skku, project = skku-students'

In [27]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import csv
import json
from os.path import exists
import math

In [20]:
"""
Building Quantum Circuit Part
"""


def buildQAOA(dataset, parameters):
    num = len(dataset)
    weights = calculateWeights(dataset)
    p = len(parameters)//2
    beta = parameters[:p]
    gamma = parameters[p:]
    
    # Init Quantum Circuit
    qc = QuantumCircuit(num)
    
    # Mixer Ground State
    for i in range(num):
        qc.h(i)
    # Evolving
    for i in range(p):
        costFunction(qc, gamma[i], weights)
        mixerHamiltonian(qc, beta[i])

    qc.measure_all()
    return (qc, weights)



def mixerHamiltonian(qc, beta):
    num = qc.num_qubits
    for i in range(num):
        qc.rx(2*beta, i)
    return

def calculateWeights(dataset):
    sh = np.zeros((len(dataset),len(dataset)))
    for i in np.arange(len(dataset)):
        for j in np.arange(len(dataset)):
            distance = np.sqrt((dataset[i][0]-dataset[j][0])**2+(dataset[i][1]-dataset[j][1])**2)
            sh[i][j] = distance
    return sh

    # i1, i2 = qubit index
def costFunctionUnit(qc, gamma, i1, i2, weight):
    qc.p(gamma*weight, i1)
    qc.p(gamma*weight, i2)
    qc.cp(-2*gamma*weight, i1, i2)
    return

def costFunction(qc, gamma, weights):
    num = qc.num_qubits
    for i in range(num):
        for k in range(i+1, num):
            costFunctionUnit(qc, gamma, i, k, weights[i][k])
    return

In [21]:
"""
Optimizing Parameters Part
"""


def countCost(bstring, weights):
    cost = 0
    num = len(bstring)
    for i in range(num):
        for k in range(i+1, num):
            if bstring[i] != bstring[k]:
                cost -= weights[i][k]
    return cost

def computeExpectation(counts, weights):
    total_cost = 0
    total_counts = 0
    
    for bstring, count in counts.items():
        cost = countCost(bstring, weights)
        total_cost += cost * count
        total_counts += count
        
    expectation_value = total_cost/total_counts
        
    return expectation_value

def getExpectation(dataset, backend, fname, shots=512):

#     backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def executeCircuit(parameters):
        
        qc, weights = buildQAOA(dataset, parameters)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=shots).result().get_counts()
        bstring = max(counts, key=counts.get)
        expectation_value = computeExpectation(counts, weights)
        
        tmp = {
            bstring: expectation_value
        }
        
        with open('tmp.json', 'w') as f:
            json.dump(tmp, f)
        
        return expectation_value
    
    return executeCircuit



In [47]:
"""
Execution Part
"""


def runQAOA(dataset, p, backend = None, fname = 'results', shots = 10000):
    if backend == None:
        backend = Aer.get_backend('qasm_simulator')
        
    
    expectation = getExpectation(dataset, backend, fname, shots)
#     init_parameters = np.ones(2*p) # TODO: Need Check
    init_parameters = []
    for i in range(2):
        for k in range(p):
            if i == 0:
                init_parameters.append(0.2*np.pi)
            else:
                init_parameters.append(0.1*np.pi)
            
    res = minimize(expectation, 
                          init_parameters, 
                          method='COBYLA')
    
    result = []
    with open('tmp.json', 'r') as f:
        tmp = json.load(f)
        tmp = list(zip(tmp.keys(),tmp.values()))[0]
        result.append(tmp)
        
    file_exists = exists(f'{fname}.json')
    if not file_exists:
        with open(f'{fname}.json', 'w') as f:
            empty = {fname:[]}
            json.dump(empty, f)

    """
    json_data = {
    fname: [[bstring, expectation_value], ...]
    }

    """

    with open(f'{fname}.json', 'r') as f:
        json_data = json.load(f)
        json_data[fname].extend(result)

    with open(f'{fname}.json', 'w') as f:
        json.dump(json_data, f)
    
    return res

In [48]:
"""
Experiment Part
"""

qasm_sim = Aer.get_backend('qasm_simulator')

def executeExperiment(dataset, p, fname, exp_shots, backend = qasm_sim):
    for i in range(exp_shots):
        runQAOA(dataset, p, backend, fname)
    return
    
def p_exp(dataset, backend = qasm_sim):
        n = len(dataset)
        p = math.ceil(math.log(n))
        exp_shots = n+2
        fname = "p_exp"
        for i in range(p):
            executeExperiment(dataset, i, fname, exp_shots, backend)
        return



In [49]:
executeExperiment(dataset, 1, 'test', 3) # dataset, p, fname, exp_shots, backend = qasm_sim